In [1]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [2]:
import os
if not os.path.isdir('saved_models_mnist_{}'.format('cogan')):
    os.mkdir('saved_models_mnist_{}'.format('cogan'))
f = open('saved_models_mnist_{}/log_collapse1.txt'.format('cogan'), mode='w')
import cv2
import torch.utils.data as Data

import scipy

from keras.datasets import cifar10,fashion_mnist,mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from PIL import Image
import matplotlib.pyplot as plt

import sys
import os

import numpy as np

class COGAN():
    """Reference: https://wiseodd.github.io/techblog/2017/02/18/coupled_gan/"""
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.d1, self.d2 = self.build_discriminators()
        self.d1.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d2.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.g1, self.g2 = self.build_generators()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img1 = self.g1(z)
        img2 = self.g2(z)

        # For the combined model we will only train the generators
        self.d1.trainable = False
        self.d2.trainable = False

        # The valid takes generated images as input and determines validity
        valid1 = self.d1(img1)
        valid2 = self.d2(img2)

        # The combined model  (stacked generators and discriminators)
        # Trains generators to fool discriminators
        self.combined = Model(z, [valid1, valid2])
        self.combined.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
                                    optimizer=optimizer)

    def build_generators(self):
        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img1 = model(noise)
        img2 = model(noise)



        return Model(noise, img1), Model(noise, img2)

    def build_discriminators(self):
        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img1 = Input(shape=self.img_shape)
        img2 = Input(shape=self.img_shape)
        validity1 = model(img1)
        validity2 = model(img2)

        return Model(img1, validity1), Model(img2, validity2)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (X_test, y_test) = mnist.load_data()

        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        X_test = np.expand_dims(X_test, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Images in domain A and B (rotated)
        X1 = X_train[:int(X_train.shape[0]/2)]
        X2 = X_train[int(X_train.shape[0]/2):]
        X2 = scipy.ndimage.interpolation.rotate(X2, 90, axes=(1, 2))

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X1.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                # imgs = X_train[index * batch_size:(index + 1) * batch_size]

                imgs1 = X1[index * batch_size:(index + 1) * batch_size]
                imgs2 = X2[index * batch_size:(index + 1) * batch_size]
                # print(len(imgs1))

                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, 100))

                # Generate a batch of new images
                gen_imgs1 = self.g1.predict(noise)
                gen_imgs2 = self.g2.predict(noise)

                # Train the discriminators
                d1_loss_real = self.d1.train_on_batch(imgs1, valid)
                d2_loss_real = self.d2.train_on_batch(imgs2, valid)
                d1_loss_fake = self.d1.train_on_batch(gen_imgs1, fake)
                d2_loss_fake = self.d2.train_on_batch(gen_imgs2, fake)
                d1_loss = 0.5 * np.add(d1_loss_real, d1_loss_fake)
                d2_loss = 0.5 * np.add(d2_loss_real, d2_loss_fake)

                # ------------------
                #  Train Generators
                # ------------------

                g_loss = self.combined.train_on_batch(noise, [valid, valid])

                # Plot the progress
                print("epoch:%d step:%d [D1 loss: %f, acc.: %.2f%%] [D2 loss: %f, acc.: %.2f%%] [G loss: %f]" \
                      % (epoch, global_step, d1_loss[0], 100 * d1_loss[1], d2_loss[0], 100 * d2_loss[1], g_loss[0]))

                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sample_num):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * int(c/2), 100))
        gen_imgs1 = self.g1.predict(noise)
        gen_imgs2 = self.g2.predict(noise)

        gen_imgs = np.concatenate([gen_imgs1, gen_imgs2])
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')
        

if __name__ == '__main__':
    gan = COGAN()
    gan.train(epochs=30, batch_size=64, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D1 loss: 1.411762, acc.: 38.28%] [D2 loss: 0.651540, acc.: 63.28%] [G loss: 2.298543]
epoch:0 step:2 [D1 loss: 0.184503, acc.: 92.19%] [D2 loss: 0.087614, acc.: 97.66%] [G loss: 6.766708]
epoch:0 step:3 [D1 loss: 0.177079, acc.: 90.62%] [D2 loss: 0.094375, acc.: 99.22%] [G loss: 9.737773]
epoch:0 step:4 [D1 loss: 0.637311, acc.: 67.97%] [D2 loss: 0.147858, acc.: 98.44%] [G loss: 11.987363]
epoch:0 step:5 [D1 loss: 1.514926, acc.: 36.72%] [D2 loss: 0.484225, acc.: 78.12%] [G loss: 11.551202]
epoch:0 step:6 [D1 loss: 0.578373, acc.: 65.62%] [D2 loss: 0.391366, acc.: 82.03%] [G loss: 10.581725]
epoch:0 step:7 [D1 loss: 0.202554, acc.: 92.97%] [D2 loss: 0.147096, acc.: 96.09%] [G loss: 9.741112]
epoch:0 step:8 [D1 loss: 0.168737, acc.: 94.53%] [D2 loss: 0.095026, acc.: 97.66%] [G loss: 8.868950]
epoch:0 step:9 [D1 loss: 0.108918, acc.: 98.44%] [D2 loss: 0.108163, acc.: 96.09%] [G loss: 6.954466]
epoch:0 step:10 [D1 loss: 0.049277, acc.: 98.44%] [D2 loss: 0.057535, acc.: 98.

epoch:0 step:82 [D1 loss: 0.943061, acc.: 50.78%] [D2 loss: 0.308751, acc.: 89.84%] [G loss: 5.076052]
epoch:0 step:83 [D1 loss: 1.725625, acc.: 14.84%] [D2 loss: 0.694335, acc.: 59.38%] [G loss: 3.507713]
epoch:0 step:84 [D1 loss: 1.177178, acc.: 47.66%] [D2 loss: 0.591618, acc.: 65.62%] [G loss: 1.946951]
epoch:0 step:85 [D1 loss: 0.350437, acc.: 87.50%] [D2 loss: 0.257805, acc.: 90.62%] [G loss: 2.771913]
epoch:0 step:86 [D1 loss: 0.906822, acc.: 53.91%] [D2 loss: 0.520212, acc.: 75.00%] [G loss: 4.761131]
epoch:0 step:87 [D1 loss: 1.211995, acc.: 25.78%] [D2 loss: 1.019207, acc.: 42.97%] [G loss: 2.331119]
epoch:0 step:88 [D1 loss: 0.476012, acc.: 75.00%] [D2 loss: 0.458645, acc.: 72.66%] [G loss: 1.138023]
epoch:0 step:89 [D1 loss: 0.216884, acc.: 96.88%] [D2 loss: 0.158240, acc.: 98.44%] [G loss: 1.412126]
epoch:0 step:90 [D1 loss: 0.418658, acc.: 82.03%] [D2 loss: 0.177266, acc.: 97.66%] [G loss: 2.662440]
epoch:0 step:91 [D1 loss: 0.990902, acc.: 46.88%] [D2 loss: 0.386109, acc

...

epoch:29 step:13711 [D1 loss: 1.316868, acc.: 43.75%] [D2 loss: 0.279632, acc.: 86.72%] [G loss: 4.033217]
epoch:29 step:13712 [D1 loss: 1.026701, acc.: 58.59%] [D2 loss: 0.250476, acc.: 90.62%] [G loss: 5.262924]
epoch:29 step:13713 [D1 loss: 1.170325, acc.: 52.34%] [D2 loss: 0.256327, acc.: 87.50%] [G loss: 4.233666]
epoch:29 step:13714 [D1 loss: 1.053556, acc.: 53.91%] [D2 loss: 0.347683, acc.: 85.94%] [G loss: 4.725872]
epoch:29 step:13715 [D1 loss: 0.762443, acc.: 62.50%] [D2 loss: 0.177416, acc.: 91.41%] [G loss: 4.671361]
epoch:29 step:13716 [D1 loss: 0.792825, acc.: 64.84%] [D2 loss: 0.184639, acc.: 92.97%] [G loss: 4.680923]
epoch:29 step:13717 [D1 loss: 0.617895, acc.: 74.22%] [D2 loss: 0.244238, acc.: 89.06%] [G loss: 4.956600]
epoch:29 step:13718 [D1 loss: 0.676352, acc.: 70.31%] [D2 loss: 0.337793, acc.: 85.16%] [G loss: 4.396188]
epoch:29 step:13719 [D1 loss: 0.686358, acc.: 70.31%] [D2 loss: 0.233936, acc.: 93.75%] [G loss: 4.943383]
epoch:29 step:13720 [D1 loss: 0.65565

epoch:29 step:13791 [D1 loss: 0.867083, acc.: 63.28%] [D2 loss: 0.254591, acc.: 89.84%] [G loss: 4.909085]
epoch:29 step:13792 [D1 loss: 1.428861, acc.: 50.00%] [D2 loss: 0.302193, acc.: 84.38%] [G loss: 5.768190]
epoch:29 step:13793 [D1 loss: 1.060871, acc.: 52.34%] [D2 loss: 0.293605, acc.: 85.16%] [G loss: 5.616412]
epoch:29 step:13794 [D1 loss: 1.033853, acc.: 56.25%] [D2 loss: 0.342545, acc.: 83.59%] [G loss: 4.119153]
epoch:29 step:13795 [D1 loss: 1.152424, acc.: 48.44%] [D2 loss: 0.495442, acc.: 80.47%] [G loss: 5.510361]
epoch:29 step:13796 [D1 loss: 1.026836, acc.: 57.81%] [D2 loss: 0.257001, acc.: 90.62%] [G loss: 5.245000]
epoch:29 step:13797 [D1 loss: 0.834221, acc.: 62.50%] [D2 loss: 0.321427, acc.: 86.72%] [G loss: 4.981564]
epoch:29 step:13798 [D1 loss: 0.594560, acc.: 74.22%] [D2 loss: 0.274521, acc.: 88.28%] [G loss: 5.503054]
epoch:29 step:13799 [D1 loss: 0.807769, acc.: 61.72%] [D2 loss: 0.389785, acc.: 82.81%] [G loss: 6.115327]
epoch:29 step:13800 [D1 loss: 0.67849

epoch:29 step:13869 [D1 loss: 1.144256, acc.: 47.66%] [D2 loss: 0.254272, acc.: 89.84%] [G loss: 5.338962]
epoch:29 step:13870 [D1 loss: 1.181552, acc.: 46.88%] [D2 loss: 0.270050, acc.: 89.84%] [G loss: 4.842941]
epoch:29 step:13871 [D1 loss: 0.841635, acc.: 60.16%] [D2 loss: 0.359943, acc.: 82.81%] [G loss: 5.123595]
epoch:29 step:13872 [D1 loss: 0.976015, acc.: 53.91%] [D2 loss: 0.220664, acc.: 89.06%] [G loss: 4.379390]
epoch:29 step:13873 [D1 loss: 0.943428, acc.: 59.38%] [D2 loss: 0.247809, acc.: 85.94%] [G loss: 5.547215]
epoch:29 step:13874 [D1 loss: 1.210912, acc.: 48.44%] [D2 loss: 0.366440, acc.: 82.81%] [G loss: 4.910562]
epoch:29 step:13875 [D1 loss: 0.916232, acc.: 57.81%] [D2 loss: 0.322803, acc.: 87.50%] [G loss: 4.292563]
epoch:29 step:13876 [D1 loss: 1.013347, acc.: 57.03%] [D2 loss: 0.335310, acc.: 84.38%] [G loss: 5.506207]
epoch:29 step:13877 [D1 loss: 1.164346, acc.: 58.59%] [D2 loss: 0.251859, acc.: 85.16%] [G loss: 4.976432]
epoch:29 step:13878 [D1 loss: 0.96384

epoch:29 step:13948 [D1 loss: 1.136873, acc.: 51.56%] [D2 loss: 0.256213, acc.: 89.84%] [G loss: 5.555156]
epoch:29 step:13949 [D1 loss: 1.083932, acc.: 53.91%] [D2 loss: 0.256952, acc.: 88.28%] [G loss: 5.051202]
epoch:29 step:13950 [D1 loss: 0.806278, acc.: 60.94%] [D2 loss: 0.401129, acc.: 84.38%] [G loss: 5.133295]
epoch:29 step:13951 [D1 loss: 0.873507, acc.: 64.06%] [D2 loss: 0.329811, acc.: 83.59%] [G loss: 5.068412]
epoch:29 step:13952 [D1 loss: 0.998244, acc.: 60.16%] [D2 loss: 0.167220, acc.: 94.53%] [G loss: 5.823526]
epoch:29 step:13953 [D1 loss: 0.840564, acc.: 60.94%] [D2 loss: 0.229930, acc.: 90.62%] [G loss: 4.824573]
epoch:29 step:13954 [D1 loss: 0.719930, acc.: 66.41%] [D2 loss: 0.411378, acc.: 82.81%] [G loss: 5.099142]
epoch:29 step:13955 [D1 loss: 1.197845, acc.: 54.69%] [D2 loss: 0.302956, acc.: 86.72%] [G loss: 5.117583]
epoch:29 step:13956 [D1 loss: 1.037632, acc.: 53.91%] [D2 loss: 0.243853, acc.: 91.41%] [G loss: 5.444916]
epoch:29 step:13957 [D1 loss: 0.87664

epoch:29 step:14024 [D1 loss: 1.192268, acc.: 48.44%] [D2 loss: 0.265569, acc.: 86.72%] [G loss: 5.483761]
epoch:29 step:14025 [D1 loss: 1.247341, acc.: 48.44%] [D2 loss: 0.249472, acc.: 87.50%] [G loss: 4.857013]
epoch:29 step:14026 [D1 loss: 1.272608, acc.: 52.34%] [D2 loss: 0.130491, acc.: 93.75%] [G loss: 5.195186]
epoch:29 step:14027 [D1 loss: 1.238864, acc.: 50.78%] [D2 loss: 0.248170, acc.: 90.62%] [G loss: 5.353874]
epoch:29 step:14028 [D1 loss: 1.421903, acc.: 53.12%] [D2 loss: 0.238002, acc.: 91.41%] [G loss: 4.615874]
epoch:29 step:14029 [D1 loss: 1.213353, acc.: 53.12%] [D2 loss: 0.234738, acc.: 91.41%] [G loss: 5.618061]
epoch:29 step:14030 [D1 loss: 0.933407, acc.: 60.16%] [D2 loss: 0.279447, acc.: 86.72%] [G loss: 5.283791]
epoch:29 step:14031 [D1 loss: 0.977790, acc.: 58.59%] [D2 loss: 0.290108, acc.: 85.94%] [G loss: 4.616015]
epoch:29 step:14032 [D1 loss: 1.086157, acc.: 56.25%] [D2 loss: 0.436761, acc.: 81.25%] [G loss: 6.034831]
epoch:29 step:14033 [D1 loss: 1.04225